---

# Chapter 2: Calculus - The Engine of Learning

---

If linear algebra is the skeleton of ML, calculus is the muscle that drives learning. The process of **training** a model is fundamentally about finding parameters that minimize a loss function—this is **optimization**, which relies entirely on derivatives.

> **Key Insight**: The gradient always points toward the direction of steepest increase. To minimize, we move in the *opposite* direction.

## 2.1 Gradients and the Jacobian Matrix

### Theory

For a scalar function $f: \mathbb{R}^n \to \mathbb{R}$, the **gradient** is the vector of partial derivatives:

$$\nabla f = \begin{bmatrix} \frac{\partial f}{\partial x_1} \\ \frac{\partial f}{\partial x_2} \\ \vdots \\ \frac{\partial f}{\partial x_n} \end{bmatrix}$$

For a **vector-valued function** $\mathbf{f}: \mathbb{R}^n \to \mathbb{R}^m$, we use the **Jacobian matrix**:

$$J = \begin{bmatrix} \frac{\partial f_1}{\partial x_1} & \cdots & \frac{\partial f_1}{\partial x_n} \\ \vdots & \ddots & \vdots \\ \frac{\partial f_m}{\partial x_1} & \cdots & \frac{\partial f_m}{\partial x_n} \end{bmatrix}$$

### 📐 Mathematical Example

**Function**: $f(x, y) = x^2 + xy + y^2$

**Partial derivatives**:
- $\frac{\partial f}{\partial x} = 2x + y$
- $\frac{\partial f}{\partial y} = x + 2y$

**At point $(x, y) = (1, 2)$**:

$$\nabla f(1, 2) = \begin{bmatrix} 2(1) + 2 \\ 1 + 2(2) \end{bmatrix} = \begin{bmatrix} 4 \\ 5 \end{bmatrix}$$

The gradient magnitude: $\|\nabla f\| = \sqrt{16 + 25} = \sqrt{41} \approx 6.4$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Tuple

# ============================================
# 2.1 GRADIENT COMPUTATION
# ============================================

def numerical_gradient(f: Callable, x: np.ndarray, h: float = 1e-5) -> np.ndarray:
    """
    Compute gradient numerically using central differences.
    
    Args:
        f: Scalar function
        x: Point to evaluate gradient
        h: Step size for finite differences
    
    Returns:
        Gradient vector
    """
    grad = np.zeros_like(x, dtype=float)
    for i in range(len(x)):
        x_plus = x.copy()
        x_minus = x.copy()
        x_plus[i] += h
        x_minus[i] -= h
        grad[i] = (f(x_plus) - f(x_minus)) / (2 * h)
    return grad


def analytical_gradient(x: np.ndarray) -> np.ndarray:
    """Analytical gradient of f(x,y) = x^2 + xy + y^2"""
    return np.array([2*x[0] + x[1], x[0] + 2*x[1]])


# Define our function
def f(x):
    return x[0]**2 + x[0]*x[1] + x[1]**2


# Test at point (1, 2)
point = np.array([1.0, 2.0])

print("="*50)
print("GRADIENT COMPUTATION")
print("="*50)
print(f"Function: f(x,y) = x² + xy + y²")
print(f"Point: ({point[0]}, {point[1]})")
print(f"f({point[0]}, {point[1]}) = {f(point)}")
print(f"\nNumerical gradient:  {numerical_gradient(f, point)}")
print(f"Analytical gradient: {analytical_gradient(point)}")
print(f"\nGradient magnitude: {np.linalg.norm(analytical_gradient(point)):.4f}")

In [ ]:
# Visualization: Gradient field
x = np.linspace(-3, 3, 20)
y = np.linspace(-3, 3, 20)
X, Y = np.meshgrid(x, y)
Z = X**2 + X*Y + Y**2

# Gradient components
U = 2*X + Y  # df/dx
V = X + 2*Y  # df/dy

plt.figure(figsize=(10, 8))
plt.contour(X, Y, Z, levels=20, cmap='viridis', alpha=0.7)
plt.colorbar(label='f(x, y)')
plt.quiver(X, Y, U, V, color='red', alpha=0.6)
plt.scatter([1], [2], c='yellow', s=200, marker='*', edgecolors='black', zorder=5)
plt.annotate('Point (1,2)', (1.1, 2.2), fontsize=12, fontweight='bold')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Gradient Field of f(x,y) = x² + xy + y²', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

print("🔴 Red arrows show gradient direction (steepest ascent)")
print("📉 To minimize, move OPPOSITE to the gradient!")

## 2.2 Backpropagation via Chain Rule

### Theory

Neural networks are **composite functions**. If we have layers:

$$L = \text{Loss}(f_3(f_2(f_1(x))))$$

The **chain rule** lets us compute derivatives through the composition:

$$\frac{\partial L}{\partial x} = \frac{\partial L}{\partial f_3} \cdot \frac{\partial f_3}{\partial f_2} \cdot \frac{\partial f_2}{\partial f_1} \cdot \frac{\partial f_1}{\partial x}$$

### 📐 Mathematical Example: 2-Layer Network

**Forward pass**:
- Input: $x = 2$
- **Layer 1**: $h = wx = 3 \times 2 = 6$ (weight $w = 3$)
- **Activation**: $a = \text{ReLU}(h) = \max(0, 6) = 6$
- **Layer 2**: $\hat{y} = va = 0.5 \times 6 = 3$ (weight $v = 0.5$)
- **Loss**: $L = (\hat{y} - y)^2 = (3 - 4)^2 = 1$ (target $y = 4$)

**Backward pass**:

$$\frac{\partial L}{\partial \hat{y}} = 2(\hat{y} - y) = 2(3 - 4) = -2$$

$$\frac{\partial L}{\partial v} = \frac{\partial L}{\partial \hat{y}} \cdot \frac{\partial \hat{y}}{\partial v} = -2 \times a = -2 \times 6 = -12$$

$$\frac{\partial L}{\partial a} = \frac{\partial L}{\partial \hat{y}} \cdot \frac{\partial \hat{y}}{\partial a} = -2 \times v = -2 \times 0.5 = -1$$

$$\frac{\partial L}{\partial w} = \frac{\partial L}{\partial a} \cdot \frac{\partial a}{\partial h} \cdot \frac{\partial h}{\partial w} = -1 \times 1 \times x = -1 \times 1 \times 2 = -2$$

In [ ]:
# ============================================
# 2.2 BACKPROPAGATION FROM SCRATCH
# ============================================

class SimpleNeuron:
    """A simple 2-layer network demonstrating backpropagation."""
    
    def __init__(self):
        self.w = 3.0  # Layer 1 weight
        self.v = 0.5  # Layer 2 weight
        
    def forward(self, x: float) -> Tuple[float, dict]:
        """Forward pass with caching for backprop."""
        h = self.w * x           # Linear layer 1
        a = max(0, h)            # ReLU activation
        y_hat = self.v * a       # Linear layer 2
        
        # Cache for backward pass
        cache = {'x': x, 'h': h, 'a': a, 'y_hat': y_hat}
        return y_hat, cache
    
    def backward(self, y_true: float, cache: dict) -> dict:
        """Backward pass computing all gradients."""
        x, h, a, y_hat = cache['x'], cache['h'], cache['a'], cache['y_hat']
        
        # Loss: L = (y_hat - y_true)^2
        dL_dy_hat = 2 * (y_hat - y_true)
        
        # Layer 2: y_hat = v * a
        dL_dv = dL_dy_hat * a
        dL_da = dL_dy_hat * self.v
        
        # ReLU: a = max(0, h)
        dL_dh = dL_da * (1 if h > 0 else 0)
        
        # Layer 1: h = w * x
        dL_dw = dL_dh * x
        dL_dx = dL_dh * self.w
        
        return {
            'dL/dy_hat': dL_dy_hat,
            'dL/dv': dL_dv,
            'dL/da': dL_da,
            'dL/dh': dL_dh,
            'dL/dw': dL_dw,
            'dL/dx': dL_dx,
        }


# Example from theory
net = SimpleNeuron()
x, y_true = 2.0, 4.0

print("="*60)
print("BACKPROPAGATION STEP-BY-STEP")
print("="*60)

# Forward pass
y_hat, cache = net.forward(x)
loss = (y_hat - y_true) ** 2

print("📥 FORWARD PASS:")
print(f"  Input x = {x}")
print(f"  h = w*x = {net.w}*{x} = {cache['h']}")
print(f"  a = ReLU(h) = {cache['a']}")
print(f"  ŷ = v*a = {net.v}*{cache['a']} = {y_hat}")
print(f"  Loss L = (ŷ - y)² = ({y_hat} - {y_true})² = {loss}")

# Backward pass
grads = net.backward(y_true, cache)

print("\n📤 BACKWARD PASS (Chain Rule):")
for name, value in grads.items():
    print(f"  {name} = {value}")

print("\n✅ These gradients tell us how to update w and v to reduce loss!")

## 2.3 Self-Attention Mechanism (Transformers)

### Theory

The **Self-Attention** mechanism is the heart of Transformers (GPT, BERT). It computes relationships between all positions in a sequence using:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

Where:
- $Q$ (Query): "What am I looking for?"
- $K$ (Key): "What do I contain?"
- $V$ (Value): "What information do I provide?"
- $d_k$: Dimension of keys (for scaling)

### Why $\sqrt{d_k}$?

The dot product $QK^T$ grows with dimension $d_k$. Large values cause softmax to saturate (output near 0 or 1), leading to **vanishing gradients**. Scaling by $\sqrt{d_k}$ keeps values in a good range.

### 📐 Mathematical Example: 3-Word Sentence

**Sentence**: "The cat sat"

Suppose each word has a 4-dimensional embedding:
- "The" → $[0.1, 0.2, 0.1, 0.3]$
- "cat" → $[0.5, 0.4, 0.6, 0.2]$
- "sat" → $[0.3, 0.5, 0.2, 0.4]$

We compute Q, K, V by multiplying with learned weight matrices.

In [ ]:
# ============================================
# 2.3 SELF-ATTENTION FROM SCRATCH
# ============================================

def softmax(x: np.ndarray) -> np.ndarray:
    """Numerically stable softmax."""
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)


def scaled_dot_product_attention(Q: np.ndarray, K: np.ndarray, V: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Compute scaled dot-product attention.
    
    Args:
        Q: Query matrix (seq_len, d_k)
        K: Key matrix (seq_len, d_k)
        V: Value matrix (seq_len, d_v)
    
    Returns:
        output: Attention output
        attention_weights: Attention weights matrix
    """
    d_k = Q.shape[-1]
    
    # Step 1: Compute attention scores
    scores = Q @ K.T  # (seq_len, seq_len)
    
    # Step 2: Scale by sqrt(d_k)
    scaled_scores = scores / np.sqrt(d_k)
    
    # Step 3: Apply softmax
    attention_weights = softmax(scaled_scores)
    
    # Step 4: Weighted sum of values
    output = attention_weights @ V
    
    return output, attention_weights


# Example: 3-word sentence with 4-dim embeddings
np.random.seed(42)

# Word embeddings (in practice, these come from an embedding layer)
X = np.array([
    [0.1, 0.2, 0.1, 0.3],  # "The"
    [0.5, 0.4, 0.6, 0.2],  # "cat"
    [0.3, 0.5, 0.2, 0.4],  # "sat"
])

# Weight matrices (in practice, these are learned)
d_model = 4
d_k = 3  # Query/Key dimension
d_v = 3  # Value dimension

W_Q = np.random.randn(d_model, d_k) * 0.5
W_K = np.random.randn(d_model, d_k) * 0.5
W_V = np.random.randn(d_model, d_v) * 0.5

# Compute Q, K, V
Q = X @ W_Q
K = X @ W_K
V = X @ W_V

print("="*60)
print("SELF-ATTENTION MECHANISM")
print("="*60)
print("Sentence: ['The', 'cat', 'sat']")
print(f"\nInput embeddings X (3 words × 4 dims):\n{X}")

# Compute attention
output, attn_weights = scaled_dot_product_attention(Q, K, V)

print(f"\n📊 Attention Weights (each row sums to 1):")
print(f"          The    cat    sat")
for i, word in enumerate(['The', 'cat', 'sat']):
    print(f"{word:>6}:  {attn_weights[i]}")

print(f"\n✅ Row sums (should be 1.0): {attn_weights.sum(axis=1)}")

In [ ]:
# Visualization of attention weights
import seaborn as sns

words = ['The', 'cat', 'sat']

plt.figure(figsize=(6, 5))
sns.heatmap(attn_weights, annot=True, fmt='.3f', 
            xticklabels=words, yticklabels=words,
            cmap='Blues', cbar_kws={'label': 'Attention Weight'})
plt.xlabel('Key (attending to)')
plt.ylabel('Query (from)')
plt.title('Self-Attention Weights\n"How much does each word attend to others?"', fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 Each word (Query) distributes its attention across all words (Keys).")
print("   The attention weights determine how much information flows from each position.")

---

# Chapter 3: Optimization - Finding the Best Solution

---

Once we have a model and a loss function, the goal is to find parameters that minimize the loss. This is **optimization**.

> **Key Insight**: Convex problems have a single global minimum. Non-convex problems (deep learning) have many local minima, but good optimizers find good solutions anyway.

## 3.1 Convex Optimization and Lagrange Multipliers

### Theory

**Convex optimization** is the "easy" case: any local minimum is a global minimum. Many classical ML algorithms (linear regression, SVM, logistic regression) are convex.

For **constrained optimization**, we use **Lagrange multipliers**. Convert:

$$\min_x f(x) \quad \text{subject to} \quad g(x) = 0$$

Into the **Lagrangian**:

$$\mathcal{L}(x, \lambda) = f(x) + \lambda g(x)$$

Then solve:
$$\nabla_x \mathcal{L} = 0 \quad \text{and} \quad \nabla_\lambda \mathcal{L} = 0$$

### 📐 Mathematical Example

**Minimize** $f(x, y) = x^2 + y^2$ **subject to** $x + y = 1$

**Lagrangian**:
$$\mathcal{L} = x^2 + y^2 + \lambda(x + y - 1)$$

**Solve**:
- $\frac{\partial \mathcal{L}}{\partial x} = 2x + \lambda = 0 \Rightarrow x = -\lambda/2$
- $\frac{\partial \mathcal{L}}{\partial y} = 2y + \lambda = 0 \Rightarrow y = -\lambda/2$
- $\frac{\partial \mathcal{L}}{\partial \lambda} = x + y - 1 = 0$

From equations 1 and 2: $x = y$. From equation 3: $2x = 1 \Rightarrow x = y = 0.5$

**Solution**: $(x^*, y^*) = (0.5, 0.5)$ with $f^* = 0.5$

In [ ]:
# ============================================
# 3.1 LAGRANGE MULTIPLIERS VISUALIZATION
# ============================================

from scipy.optimize import minimize

# Objective function
def objective(vars):
    x, y = vars
    return x**2 + y**2

# Constraint: x + y = 1 (equality, so we write it as x + y - 1 = 0)
constraint = {'type': 'eq', 'fun': lambda vars: vars[0] + vars[1] - 1}

# Solve
result = minimize(objective, [0, 0], constraints=constraint)

print("="*60)
print("LAGRANGE MULTIPLIERS EXAMPLE")
print("="*60)
print(f"Minimize: f(x,y) = x² + y²")
print(f"Subject to: x + y = 1")
print(f"\nSolution: x* = {result.x[0]:.4f}, y* = {result.x[1]:.4f}")
print(f"Optimal value: f* = {result.fun:.4f}")

# Visualization
fig, ax = plt.subplots(figsize=(8, 8))

# Contours of objective function
x = np.linspace(-1, 2, 100)
y = np.linspace(-1, 2, 100)
X, Y = np.meshgrid(x, y)
Z = X**2 + Y**2

contours = ax.contour(X, Y, Z, levels=20, cmap='viridis')
ax.clabel(contours, inline=True, fontsize=8)

# Constraint line
ax.plot(x, 1 - x, 'r-', linewidth=2, label='Constraint: x + y = 1')

# Optimal point
ax.scatter([0.5], [0.5], c='red', s=200, marker='*', 
           edgecolors='black', zorder=5, label=f'Optimal: (0.5, 0.5)')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Constrained Optimization with Lagrange Multipliers', fontweight='bold')
ax.legend()
ax.set_xlim(-0.5, 1.5)
ax.set_ylim(-0.5, 1.5)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
plt.show()

print("\n⭐ The optimal point is where the constraint line is TANGENT to a contour.")

## 3.2 SVM and the Dual Formulation

### Theory

**Support Vector Machines (SVM)** find the maximum-margin hyperplane. The **primal** problem:

$$\min_{w, b} \frac{1}{2}\|w\|^2 \quad \text{s.t.} \quad y_i(w^T x_i + b) \geq 1$$

Using Lagrange multipliers, we get the **dual** problem:

$$\max_\alpha \sum_i \alpha_i - \frac{1}{2} \sum_{i,j} \alpha_i \alpha_j y_i y_j K(x_i, x_j)$$

**Key insight**: The dual form uses only **dot products** $K(x_i, x_j) = x_i^T x_j$, which can be replaced by any kernel function (the **kernel trick**).

### 📐 Mathematical Example

**Data** (simple 2D case):
- Class +1: $(1, 2), (2, 3)$
- Class -1: $(0, 0), (1, 0)$

The dual problem becomes a **Quadratic Programming** problem.

In [ ]:
# ============================================
# 3.2 SVM FROM SCRATCH (SIMPLIFIED)
# ============================================

from scipy.optimize import minimize

class SimpleSVM:
    """Simplified SVM using scipy optimization."""
    
    def __init__(self, C: float = 1.0):
        self.C = C
        
    def fit(self, X: np.ndarray, y: np.ndarray):
        n_samples, n_features = X.shape
        
        # Compute Gram matrix K[i,j] = x_i · x_j
        K = X @ X.T
        
        # Dual objective (to maximize, so we negate for minimization)
        def dual_objective(alpha):
            return 0.5 * np.sum((alpha * y)[:, None] * (alpha * y)[None, :] * K) - np.sum(alpha)
        
        # Gradients
        def dual_gradient(alpha):
            return (alpha * y) @ (K * (y[:, None] * y[None, :])) - 1
        
        # Constraints: 0 <= alpha <= C and sum(alpha * y) = 0
        constraints = [
            {'type': 'eq', 'fun': lambda a: np.dot(a, y)}
        ]
        bounds = [(0, self.C) for _ in range(n_samples)]
        
        # Solve
        result = minimize(
            dual_objective,
            np.zeros(n_samples),
            jac=dual_gradient,
            bounds=bounds,
            constraints=constraints,
            method='SLSQP'
        )
        
        self.alpha = result.x
        
        # Find support vectors (alpha > threshold)
        sv_mask = self.alpha > 1e-5
        self.support_vectors = X[sv_mask]
        self.support_labels = y[sv_mask]
        self.support_alphas = self.alpha[sv_mask]
        
        # Compute w = sum(alpha_i * y_i * x_i)
        self.w = np.sum((self.alpha * y)[:, None] * X, axis=0)
        
        # Compute b using support vectors
        self.b = np.mean(self.support_labels - self.support_vectors @ self.w)
        
        return self
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        return np.sign(X @ self.w + self.b)


# Example data
X = np.array([
    [1, 2],   # +1
    [2, 3],   # +1  
    [0, 0],   # -1
    [1, 0],   # -1
], dtype=float)
y = np.array([1, 1, -1, -1], dtype=float)

# Train SVM
svm = SimpleSVM(C=100.0)
svm.fit(X, y)

print("="*60)
print("SUPPORT VECTOR MACHINE (DUAL FORM)")
print("="*60)
print(f"Data points: {len(X)}")
print(f"Lagrange multipliers (α): {svm.alpha}")
print(f"\nSupport vectors (α > 0):")
for sv, label, alpha in zip(svm.support_vectors, svm.support_labels, svm.support_alphas):
    print(f"  {sv} (y={int(label)}, α={alpha:.4f})")
print(f"\nWeight vector w: {svm.w}")
print(f"Bias b: {svm.b:.4f}")
print(f"\nDecision boundary: {svm.w[0]:.3f}x + {svm.w[1]:.3f}y + {svm.b:.3f} = 0")

In [ ]:
# Visualization
plt.figure(figsize=(8, 6))

# Plot data points
plt.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', s=100, label='Class +1', edgecolors='black')
plt.scatter(X[y == -1, 0], X[y == -1, 1], c='red', s=100, label='Class -1', edgecolors='black')

# Plot support vectors
plt.scatter(svm.support_vectors[:, 0], svm.support_vectors[:, 1], 
            s=200, facecolors='none', edgecolors='green', linewidths=2, label='Support Vectors')

# Plot decision boundary and margins
x_range = np.linspace(-0.5, 3, 100)
y_boundary = -(svm.w[0] * x_range + svm.b) / svm.w[1]
y_margin_pos = -(svm.w[0] * x_range + svm.b - 1) / svm.w[1]
y_margin_neg = -(svm.w[0] * x_range + svm.b + 1) / svm.w[1]

plt.plot(x_range, y_boundary, 'k-', linewidth=2, label='Decision Boundary')
plt.plot(x_range, y_margin_pos, 'k--', linewidth=1)
plt.plot(x_range, y_margin_neg, 'k--', linewidth=1)
plt.fill_between(x_range, y_margin_neg, y_margin_pos, alpha=0.1, color='yellow')

plt.xlabel('x₁')
plt.ylabel('x₂')
plt.title('SVM: Maximum Margin Classifier', fontweight='bold')
plt.legend(loc='upper left')
plt.xlim(-0.5, 3)
plt.ylim(-1, 4)
plt.grid(True, alpha=0.3)
plt.show()

print("💡 The margin (yellow area) is maximized.")
print("   Only support vectors determine the decision boundary!")

## 3.3 ADMM: Industrial-Scale Optimization (Uber Case)

### Theory

**Alternating Direction Method of Multipliers (ADMM)** solves problems of the form:

$$\min_x f(x) + g(z) \quad \text{s.t.} \quad Ax + Bz = c$$

The algorithm alternates between:

1. **x-update**: $x^{k+1} = \arg\min_x (f(x) + \frac{\rho}{2}\|Ax + Bz^k - c + u^k\|_2^2)$
2. **z-update**: $z^{k+1} = \arg\min_z (g(z) + \frac{\rho}{2}\|Ax^{k+1} + Bz - c + u^k\|_2^2)$
3. **Dual update**: $u^{k+1} = u^k + Ax^{k+1} + Bz^{k+1} - c$

### 🏭 Industrial Application: Uber

Uber uses ADMM for budget allocation across cities. Each city solves a local problem, while the global constraint ensures total budget is respected.

### 📐 Mathematical Example: LASSO Regression

$$\min_x \frac{1}{2}\|Ax - b\|_2^2 + \lambda\|x\|_1$$

We introduce $z = x$ and solve using ADMM.

In [ ]:
# ============================================
# 3.3 ADMM FOR LASSO
# ============================================

def soft_threshold(x: np.ndarray, threshold: float) -> np.ndarray:
    """Soft thresholding operator (proximal operator for L1)."""
    return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)


def admm_lasso(A: np.ndarray, b: np.ndarray, lam: float, 
               rho: float = 1.0, max_iter: int = 100, tol: float = 1e-6):
    """
    Solve LASSO using ADMM.
    
    min (1/2)||Ax - b||² + λ||x||₁
    """
    n = A.shape[1]
    
    # Initialize
    x = np.zeros(n)
    z = np.zeros(n)
    u = np.zeros(n)
    
    # Precompute (A^T A + ρI)^{-1} A^T b for efficiency
    AtA = A.T @ A
    Atb = A.T @ b
    L = AtA + rho * np.eye(n)
    L_inv = np.linalg.inv(L)
    
    history = []
    
    for k in range(max_iter):
        # x-update: solve (A^T A + ρI)x = A^T b + ρ(z - u)
        x = L_inv @ (Atb + rho * (z - u))
        
        # z-update: soft thresholding
        z_new = soft_threshold(x + u, lam / rho)
        
        # u-update (dual variable)
        u = u + x - z_new
        
        # Check convergence
        primal_residual = np.linalg.norm(x - z_new)
        history.append(primal_residual)
        
        z = z_new
        
        if primal_residual < tol:
            break
    
    return x, history


# Generate example data
np.random.seed(42)
n_samples, n_features = 50, 20
A = np.random.randn(n_samples, n_features)
true_x = np.zeros(n_features)
true_x[:5] = [3, -2, 1.5, -1, 0.5]  # Only 5 non-zero coefficients
b = A @ true_x + 0.1 * np.random.randn(n_samples)

# Solve LASSO
lam = 0.5
x_lasso, history = admm_lasso(A, b, lam, rho=1.0, max_iter=200)

print("="*60)
print("ADMM FOR LASSO REGRESSION")
print("="*60)
print(f"Problem: min (1/2)||Ax - b||² + {lam}||x||₁")
print(f"True sparse coefficients:   {true_x[:8]}...")
print(f"LASSO solution (rounded):   {np.round(x_lasso[:8], 3)}...")
print(f"\nConverged in {len(history)} iterations")
print(f"Non-zero coefficients: {np.sum(np.abs(x_lasso) > 0.01)} (true: 5)")